<a href="https://colab.research.google.com/github/KeoratileShongoane/AML/blob/main/Spark_(1)_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***
# **Data Science in Practise**: Assignment 2
#### __Name:__ Keoratile Shongoane
#### __Student number:__ 1389986

***


# __Imports__

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=1e0b759ed8a5f6bf04aa22b9f5cfae24b9b0a28aecd7f62e792669e5957f9ce2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


The cell below imports spark, starts a spark session, and then prints out the details of the running spark instance.

In [2]:
# Import SparkSession
from pyspark.sql import SparkSession, column
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
import numpy as np
import pandas as pd



# __1. Starting Spark Session__

In [3]:
# Create a Spark Session
spark = SparkSession.builder.appName('ml-income').getOrCreate()

# __2. Proprocessing__

### __2.1 Importing Data__

In [23]:
df = spark.read.csv('/content/income.csv', header=True, nullValue='?',ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True, inferSchema=True)



### __2.2 Viewing data types__

In [5]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_years: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- citizenship: string (nullable = true)
 |-- income_class: string (nullable = true)



### __2.3 Dropping missing values__

In [6]:
df = df.na.drop()
df.show()

+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+
|age|       workclass|weight|   education|education_years|      marital_status|       occupation| relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|  citizenship|income_class|
+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+
| 39|       State-gov| 77516|   Bachelors|             13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40|United-States|       <=50K|
| 50|Self-emp-not-inc| 83311|   Bachelors|             13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Male|           0|           0|            1

### __2.4 Transposing Data__

In [7]:

pd.DataFrame(df.take(5),columns=df.columns).transpose()

,0,1,2,3,4
age,39,50,38,53,28
workclass,State-gov,Self-emp-not-inc,Private,Private,Private
weight,77516,83311,215646,234721,338409
education,Bachelors,Bachelors,HS-grad,11th,Bachelors
education_years,13,13,9,7,13
marital_status,Never-married,Married-civ-spouse,Divorced,Married-civ-spouse,Married-civ-spouse
occupation,Adm-clerical,Exec-managerial,Handlers-cleaners,Handlers-cleaners,Prof-specialty
relationship,Not-in-family,Husband,Not-in-family,Husband,Wife
race,White,White,White,Black,Black
sex,Male,Male,Male,Male,Female


### __2.5 Calculating Descriptive statistics__

In [8]:
numeric_features = [t[0] for t in df.dtypes if t[1] == 'int']
df.select(numeric_features).describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
age,30162,38.437901995888865,13.134664776856017,17,90
weight,30162,189793.83393011073,105652.97152851892,13769,1484705
education_years,30162,10.12131158411246,2.5499949188567355,1,16
capital_gain,30162,1092.0078575691268,7406.346496683523,0,99999
capital_loss,30162,88.37248856176646,404.2983704863746,0,4356
hours_per_week,30162,40.93123798156621,11.97998422927489,1,99


 ### __2.6 Encoding__

In [9]:
label_stringIdx= StringIndexer(inputCols=["workclass", "education", "marital_status","occupation", "relationship", "race","sex","citizenship"], outputCols=["label_workclass", "label_education", "label_marital_status", "label_occupation", "label_relationship","label_race", "label_sex", "label_citizenship"])
df = label_stringIdx.fit(df).transform(df)
df.show()

+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+------------------+----------+---------+-----------------+
|age|       workclass|weight|   education|education_years|      marital_status|       occupation| relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|  citizenship|income_class|label_workclass|label_education|label_marital_status|label_occupation|label_relationship|label_race|label_sex|label_citizenship|
+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+------------------+----------+---------+-----------------

### __2.7 Transforming Features__

In [10]:
#using the vector assember to feature transform

numericCols =["label_workclass", "label_education", "label_marital_status", "label_occupation", "label_relationship","label_race", "label_sex", "label_citizenship"]
assembler = VectorAssembler(inputCols=numericCols, outputCol="features")
df = assembler.transform(df)
df.show()

+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+------------------+----------+---------+-----------------+--------------------+
|age|       workclass|weight|   education|education_years|      marital_status|       occupation| relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|  citizenship|income_class|label_workclass|label_education|label_marital_status|label_occupation|label_relationship|label_race|label_sex|label_citizenship|            features|
+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+---------------

### __2.8 Encoding Target Column__

In [11]:
labels = StringIndexer(inputCol="income_class", outputCol="label")
df = labels.fit(df).transform(df)
df.show()

+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+------------------+----------+---------+-----------------+--------------------+-----+
|age|       workclass|weight|   education|education_years|      marital_status|       occupation| relationship|              race|   sex|capital_gain|capital_loss|hours_per_week|  citizenship|income_class|label_workclass|label_education|label_marital_status|label_occupation|label_relationship|label_race|label_sex|label_citizenship|            features|label|
+---+----------------+------+------------+---------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+-------------+------------+---------------+---------------+--------------------+----------------+---

In [12]:
# pd.DataFrame(df.take(100),columns=df.columns)

### __2.9 Train-Test Split__

In [13]:
#splitting our datasets
train, test = df.randomSplit([0.7,0.3])


In [14]:
print("Training Dataset Count: " + str(train.count()))
print("Test Dataset Count: " + str(test.count()))

Training Dataset Count: 21131
Test Dataset Count: 9031


# __3. Model Training__

#### Here, I train 2 models, namely: `Decision Tree Classifier` and `Random Forest Classifier`.

### __3.1 Random Forest Classifier__

In [15]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features", maxBins=41)
modelrf = rf.fit(train)

In [16]:
#The RandomnForest Predictor
predictionsrf = modelrf.transform(test)
predictionsrf.select("features", "label").show(25)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.0,5.0,1.0,0.0,...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|[0.0,7.0,1.0,5.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|[0.0,5.0,1.0,3.0,...|  0.0|
|(8,[1,2,3,4],[5.0...|  0.0|
|(8,[1,2,4,6],[5.0...|  0.0|
|(8,[1,2,3,4],[5.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|[0.0,10.0,1.0,5.0...|  0.0|
|[0.0,11.0,1.0,4.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,8.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[1.0...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|[0.0,5.0,1.0,4.0,...|  0.0|
|[0.0,5.0,1.0,4.0,...|  0.0|
+--------------------+-----+
only showing top 25 rows



### __3.2 Decision Tree Classifier__

In [17]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=41)
modeldt = dt.fit(train)

In [18]:
#the DecisionTree Prediction
predictionsdt = modeldt.transform(test)
predictionsdt.select("features", "label").show(25)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[2.0,5.0,1.0,0.0,...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|[0.0,7.0,1.0,5.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|[0.0,5.0,1.0,3.0,...|  0.0|
|(8,[1,2,3,4],[5.0...|  0.0|
|(8,[1,2,4,6],[5.0...|  0.0|
|(8,[1,2,3,4],[5.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|[0.0,10.0,1.0,5.0...|  0.0|
|[0.0,11.0,1.0,4.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,8.0,...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|(8,[1,2,3,4],[1.0...|  0.0|
|(8,[1,2,3,4],[11....|  0.0|
|(8,[1,2,3,4],[7.0...|  0.0|
|[0.0,5.0,1.0,5.0,...|  0.0|
|[0.0,5.0,1.0,4.0,...|  0.0|
|[0.0,5.0,1.0,4.0,...|  0.0|
+--------------------+-----+
only showing top 25 rows



# __4. Model Evaluation__

### __4.1 Random Forest Classifier__

In [19]:
# Initialize evaluators for RandomForest
evaluator_accuracy_rf = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision_rf = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall_rf = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1_rf = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Evaluate metrics for RandomForest
accuracy_rf = evaluator_accuracy_rf.evaluate(predictionsrf)
precision_rf = evaluator_precision_rf.evaluate(predictionsrf)
recall_rf = evaluator_recall_rf.evaluate(predictionsrf)
f1_rf = evaluator_f1_rf.evaluate(predictionsrf)

print("RandomForest Metrics:\n")
print(f"Accuracy = {accuracy_rf}")
print(f"Precision = {precision_rf}")
print(f"Recall = {recall_rf}")
print(f"F1 Score = {f1_rf}")
print(f"Test Error = {1.0 - accuracy_rf}")


RandomForest Metrics:

Accuracy = 0.8124238733252132
Precision = 0.7995818278232728
Recall = 0.8124238733252132
F1 Score = 0.7969625776647076
Test Error = 0.18757612667478685


In [20]:
#confusionmetric
preds = predictionsrf.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType()))

preds = preds.select(['prediction', 'label'])
metrics = MulticlassMetrics(preds.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[6346.  440.]
 [1254.  991.]]


### __4.2 Decision Tree Classifier__

In [21]:

# Initialize evaluators for DecisionTree
evaluator_accuracy_dt = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_precision_dt = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall_dt = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
evaluator_f1_dt = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

# Evaluate metrics for DecisionTree
accuracy_dt = evaluator_accuracy_dt.evaluate(predictionsdt)
precision_dt = evaluator_precision_dt.evaluate(predictionsdt)
recall_dt = evaluator_recall_dt.evaluate(predictionsdt)
f1_dt = evaluator_f1_dt.evaluate(predictionsdt)

print("DecisionTree Metrics:\n")
print(f"Accuracy = {accuracy_dt}")
print(f"Precision = {precision_dt}")
print(f"Recall = {recall_dt}")
print(f"F1 Score = {f1_dt}")
print(f"Test Error = {1.0 - accuracy_dt}")


DecisionTree Metrics:

Accuracy = 0.8133097109954601
Precision = 0.8007065871560185
Recall = 0.81330971099546
F1 Score = 0.799192301160924
Test Error = 0.18669028900453988


In [22]:
preds = predictionsdt.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType()))

preds = preds.select(['prediction', 'label'])
metrics = MulticlassMetrics(preds.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

[[6324.  462.]
 [1224. 1021.]]
